In [ ]:
import pandas as pd
import pyodbc
from datetime import datetime
import re

SERVER = r"TRC-DC-BDM\BD"
DATABASE = "SRC"

# Configuração da conexão com o banco de dados - CORRIGIDO
conn_str = (
    f'DRIVER={{SQL Server}};'
    f'SERVER={SERVER};'
    f'DATABASE={DATABASE};'
    f'Trusted_Connection=yes;'
)

# Caminho do arquivo SQL
sql_file_path = r'\\trc-dc-ad\Planejamento\MIS\CARTEIRAS\Ouze\Consultas\base_ouze.sql'

# Definir o mês desejado (setembro/2025)
dt_ini = '2025-09-01'
dt_fim = '2025-09-30'

def carregar_e_executar_sql(caminho_arquivo, data_inicio, data_fim):
    """
    Lê o arquivo SQL, substitui as datas e executa a consulta
    """
    try:
        # Ler o arquivo SQL
        with open(caminho_arquivo, 'r', encoding='utf-8') as file:
            sql_content = file.read()
        
        print(f"Arquivo SQL carregado: {caminho_arquivo}")
        
        # Substituir as datas no conteúdo do SQL
        sql_content = re.sub(
            r"DECLARE @DT_INI AS DATE = '\d{4}-\d{2}-\d{2}'",
            f"DECLARE @DT_INI AS DATE = '{data_inicio}'",
            sql_content
        )
        
        sql_content = re.sub(
            r"DECLARE @DT_FIM AS DATE = '\d{4}-\d{2}-\d{2}'",
            f"DECLARE @DT_FIM AS DATE = '{data_fim}'",
            sql_content
        )
        
        sql_content = re.sub(
            r"DECLARE @DT\s+AS DATE = '\d{4}-\d{2}-\d{2}'",
            f"DECLARE @DT AS DATE = '{data_inicio}'",
            sql_content
        )
        
        sql_content = re.sub(
            r"DECLARE @DT2\s+AS DATE = '\d{4}-\d{2}-\d{2}'",
            f"DECLARE @DT2 AS DATE = '{data_fim}'",
            sql_content
        )
        
        print(f"Datas ajustadas: {data_inicio} até {data_fim}")
        print("Conectando ao banco de dados...")
        
        # Conectar e executar
        conn = pyodbc.connect(conn_str)
        print("Conexão estabelecida. Executando consulta...")
        
        # Executar a query e carregar em DataFrame
        df_discagens = pd.read_sql(sql_content, conn)
        
        # Fechar conexão
        conn.close()
        
        print(f"\n✓ Dados carregados com sucesso!")
        print(f"Total de registros: {len(df_discagens):,}")
        print(f"\nColunas disponíveis: {list(df_discagens.columns)}")
        print(f"\nPrimeiras linhas:")
        print(df_discagens.head())
        
        return df_discagens
        
    except FileNotFoundError:
        print(f"❌ Erro: Arquivo não encontrado em {caminho_arquivo}")
        return None
    except pyodbc.Error as db_error:
        print(f"❌ Erro de banco de dados: {db_error}")
        return None
    except Exception as e:
        print(f"❌ Erro ao executar consulta: {e}")
        import traceback
        traceback.print_exc()
        return None

# Executar a função
print("Iniciando execução...")
df_discagens = carregar_e_executar_sql(sql_file_path, dt_ini, dt_fim)

# Verificar informações do DataFrame
if df_discagens is not None:
    print("\n" + "="*60)
    print("INFORMAÇÕES DO DATAFRAME")
    print("="*60)
    print(df_discagens.info())
    print(f"\nShape: {df_discagens.shape}")
else:
    print("\n❌ Falha ao carregar os dados.")

: 